In [8]:
import tensorflow as tf
from tensorflow.python.framework import ops
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, os.path
import time

data_dir = '../DataSets/FaceNet/'

In [21]:
def read_data(image_paths, image_size, batch_size, max_nrof_epochs, num_threads):
    """
    Creates Tensorflow Queue to batch load images. Applies transformations to images as they are loaded.
    :param image_paths: image paths to load
    :param image_size: size to resize images to
    :param batch_size: num of images to load in batch
    :param max_nrof_epochs: total number of epochs to read through image list
    :param num_threads: num threads to use
    :return: images and labels of batch_size
    """

    images = ops.convert_to_tensor(image_paths, dtype=tf.string)

    # Makes an input queue
    input_queue = tf.train.input_producer(images,num_epochs=max_nrof_epochs, shuffle=False)

    imgs = []
    for _ in range(num_threads):
        image = read_image_from_disk(filename=input_queue)
        image = tf.random_crop(image, size=[image_size, image_size, 3])
        image.set_shape((image_size, image_size, 3))
        image = tf.image.per_image_standardization(image)

        imgs.append(image)

    image_batch = tf.train.batch_join(imgs, batch_size=batch_size, capacity=4 * num_threads, enqueue_many=False, allow_smaller_final_batch=True)
    return image_batch


def read_image_from_disk(filename):
    """
    Consumes input tensor and loads image
    :param filename_to_label_tuple: 
    :type filename_to_label_tuple: list
    :return: tuple of image and label
    """
    file_contents = tf.read_file(filename)
    example = tf.image.decode_jpeg(file_contents, channels=3)
    return example


def get_dataset(input_directory):
    imgs = [f for f in os.listdir(input_directory)
            if os.path.isfile(input_directory+f) and f.split('.')[-1].lower() in ('jpg', 'jpeg')]
    return [(f, input_directory+f) for f in imgs]

In [22]:
import numpy as np
import tensorflow as tf

def get_embeddings(input_directory, model_fname, batch_size, num_threads):
    """
    Loads images from :param input_dir, creates embeddings using a model defined at :param model_path, and trains
     a classifier outputted to :param output_path
     
    :param input_directory: Path to directory containing pre-processed images
    :param model_path: Path to protobuf graph file for facenet model
    :param classifier_output_path: Path to write pickled classifier
    :param batch_size: Batch size to create embeddings
    :param num_threads: Number of threads to utilize for queuing
    :param num_epochs: Number of epochs for each image
    :param min_images_per_labels: Minimum number of images per class
    :param split_ratio: Ratio to split train/test dataset
    :param is_train: bool denoting if training or evaluate
    """

    start_time = time.time()
    with tf.Session(config=tf.ConfigProto(log_device_placement=False)) as sess:
        dataset = get_dataset(input_directory)
        images = _load_images(dataset, image_size=160, batch_size=batch_size, num_threads=num_threads, num_epochs=1)

        _load_model(model_fname)

        init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
        sess.run(init_op)

        images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
        embedding_layer = tf.get_default_graph().get_tensor_by_name("embeddings:0")
        phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")

        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord, sess=sess)

        emb_array, label_array = _create_embeddings(embedding_layer, images, labels, images_placeholder,
                                                    phase_train_placeholder, sess)

        coord.request_stop()
        coord.join(threads=threads)
    return emb_array

def _load_images(dataset, image_size, batch_size, num_threads, num_epochs, random_flip=False,
                            random_brightness=False, random_contrast=False):
    image_paths = [x[1] for x in dataset]
    images = read_data(image_paths, image_size, batch_size, num_epochs, num_threads)
    return images


def _load_model(model_fname):
    """
    Load frozen protobuf graph
    :param model_filepath: Path to protobuf graph
    :type model_filepath: str
    """
    #t_input = 'input:0'
    #t_output = 'MobilenetV1/Predictions/Reshape:0'
    tf_graph_def = tf.GraphDef()
    with open(model_fname, 'rb') as f:
        tf_graph_def.ParseFromString(f.read())
    tf.import_graph_def(tf_graph_def, name='')


def _create_embeddings(embedding_layer, images, images_placeholder, phase_train_placeholder, sess):
    """
    Uses model to generate embeddings from :param images.
    :param embedding_layer: 
    :param images: 
    :param images_placeholder: 
    :param phase_train_placeholder: 
    :param sess: 
    :return: (tuple): image embeddings and labels
    """
    emb_array = None
    try:
        i = 0
        while True:
            batch_images = sess.run(images)
            emb = sess.run(embedding_layer,feed_dict={images_placeholder: batch_images, phase_train_placeholder: False})

            emb_array = np.concatenate([emb_array, emb]) if emb_array is not None else emb
            i += 1

    except tf.errors.OutOfRangeError:
        pass

    return emb_array


In [23]:
res = get_embeddings(data_dir + 'proc_images/', data_dir+'facenet_20170512-110547.pb', 64,1)

TypeError: Expected string passed to parameter 'filename' of op 'ReadFile', got <tensorflow.python.ops.data_flow_ops.FIFOQueue object at 0x000001D9F0E9BC18> of type 'FIFOQueue' instead.